### Run the following chunk of code to import any libraries or packages necessary to run the rest of the code.

In [ ]:
import numpy as np
import pandas as pd
import os as os
import re

### Run the following chunk of code to extract the time averaged (TA) forces (Electrostatic & Hydrophobic) between all pairs of amino acids (AAs) in the simulation for all time steps.

The following code will calculate the TA forces between all pairs of AAs in the simulation for all time steps. Note, users will need to adjust the base_Directory variable and other directory variables to match their directory structure, or match their directory structure to that of the code.

The code can be broken into two distinct pieces. The first piece is calculating the pairwise distance between all particles in the box at all timesteps. Because this step is memory intensive, it is completed AA by AA instead of all at once. So, for each AA in the box, the positions at all times are copied to match the shape of the .npy array containing the positions of all of the amino acids at all times. This allows us to easily find the distance between all AAs relative to the AA in question.

Then, the second piece of the code is executed. In this next step, we use the pairwise distances generated in the previous step to calculate the average pairwise force between the AA in question and all other AAs in the box.

The two steps outlined are completed for each AA in the simulation box, and the time averaged pairwise forces are saved to an output array. 

In [ ]:
base_Directory = ""
phos = 0
equilibration_Time = 4000

input_Working_Directory_One = base_Directory + "Inputs/Positions/" + str(phos) + "x_Phos/"
input_Working_Directory_Two = base_Directory + "Inputs/Labels/" + str(phos) + "x_Phos/"
input_Working_Directory_Three = base_Directory + "Inputs/"
output_Working_Directory_One = base_Directory + "Outputs/Electrostatic/" + str(phos) + "x_Phos/"
output_Working_Directory_Two = base_Directory + "Outputs/Hydrophobic/" + str(phos) + "x_Phos/"
coarse_Grained_Stats = pd.read_csv(input_Working_Directory_Three + "Amino_Acid_Coarse_Grained_Stats.csv")

charges = coarse_Grained_Stats["Charge"].values
lambdas = coarse_Grained_Stats["Lambda"].values
sigmas = coarse_Grained_Stats["Sigma"].values/10 #nm

temperature = 295 #Kelvin
objects_Per_Mole = 6.02*10**(23)

boltzmann_Constant = 1.38*10**(-23) #J/K
boltzmann_Constant_Converted = boltzmann_Constant*(1/(1*10**(3)))*objects_Per_Mole # kJ/mol K

water_Dielectric_Constant = 78.4
electron_Charge = 1.602*10**(-19)
vacuum_Permittivity = 8.85*10**(-12) #C^2/(Nm^2)
vacuum_Permittivity_Converted = vacuum_Permittivity*((1/(1*10**9))*((1*10**3)/1)*(1/(objects_Per_Mole))) #C^2 mol/kJ nm
yukawa_Epsilon_Scaling = (electron_Charge)**(2)/(4*np.pi*water_Dielectric_Constant*vacuum_Permittivity_Converted) #kJ nm/mol
bjerrum_Length = (electron_Charge**2)/(4*np.pi*vacuum_Permittivity*water_Dielectric_Constant*boltzmann_Constant*temperature) #m
bjerrum_Length_Converted = bjerrum_Length*((1*10**(9))/(1)) #nm
debye_Sum_Scaling = bjerrum_Length_Converted*4*np.pi

for file in os.listdir(input_Working_Directory_One):
    print(file)
    trajectory_Position_Data = np.load(input_Working_Directory_One + file)
    trajectory_Particle_Labels = np.load(input_Working_Directory_Two + file.replace("Positions.npy","Particle_Labels.npy"))
    total_Steps = np.shape(trajectory_Position_Data)[0]
    AAs = np.shape(trajectory_Position_Data)[1]
    
    salt_Concentration = int(re.split("Protein_|_mM",file)[1])/1000 #M
    LJ_Potential_Epsilon = float(re.split("phos_|_Epsilon",file)[1])
    
    salt_Sum = salt_Concentration*(1**2) + salt_Concentration*((-1)**2)
    salt_Sum_Converted = salt_Sum*objects_Per_Mole*((1)/(1*10**(24))) #ions/nm^3
    
    monopotassium_Phosphate_Concentration = 0.006085 #mol/L
    dipotassium_Phosphate_Concentration = 0.01392 #mol/L
    buffer_Sum = dipotassium_Phosphate_Concentration*(1**2) + dipotassium_Phosphate_Concentration*(1**2) + \
    dipotassium_Phosphate_Concentration*((-1)**2) + dipotassium_Phosphate_Concentration*((-1)**2) + \
    monopotassium_Phosphate_Concentration*(1**2) + monopotassium_Phosphate_Concentration*((-1)**2) #mol/L
    buffer_Sum_Converted = buffer_Sum*objects_Per_Mole*((1)/(1*10**(24))) #ions/nm^3

    debye_Length = 1/(np.sqrt(debye_Sum_Scaling*(buffer_Sum_Converted + salt_Sum_Converted))) #nm
    
    LJ_Potential_Epsilon_Converted = LJ_Potential_Epsilon*(4.184/1) #kJ/mol
    
    average_Electrostatic_Forces_Array = np.zeros((AAs,AAs))
    average_Hydrophobic_Forces_Array = np.zeros((AAs,AAs))
    
    for AA in range(AAs):
        if AA%100 == 0:
            print(AA)
        copied_Position_Array = np.repeat(trajectory_Position_Data[equilibration_Time:,AA,:][:,None,:],AAs,axis = 1)
        distances_Array = np.linalg.norm(copied_Position_Array - trajectory_Position_Data[equilibration_Time:], axis = 2)    

        yukawa_Pairs = np.asarray(np.where((distances_Array < 3.5) & (distances_Array > 0))).T
        ashbaugh_Pairs = np.asarray(np.where((distances_Array < 2.5) & (distances_Array > 0))).T
        electrostatic_Force = 0
        ashbaugh_Force = 0

        current_Electrostatic_Forces_Array = np.zeros((1,AAs,total_Steps - equilibration_Time))
        current_Hydrophobic_Forces_Array = np.zeros((1,AAs,total_Steps - equilibration_Time))

        yukawa_Indices_One = np.asarray([AA]*len(yukawa_Pairs))
        yukawa_Indices_Two = yukawa_Pairs[:,1]
        labels_One = trajectory_Particle_Labels[0,yukawa_Indices_One]
        labels_Two = trajectory_Particle_Labels[0,yukawa_Indices_Two]
        charges_One = charges[labels_One]
        charges_Two = charges[labels_Two]

        current_Distances = distances_Array[yukawa_Pairs[:,0],yukawa_Indices_Two]

        electrostatic_Forces = (((yukawa_Epsilon_Scaling*charges_One*charges_Two)/(current_Distances))*np.exp(-(current_Distances/debye_Length)))*((1/current_Distances) + (1/debye_Length))
        current_Electrostatic_Forces_Array[0,yukawa_Indices_Two,yukawa_Pairs[:,0]] = electrostatic_Forces
        
        hydrophobic_Forces = np.zeros(np.shape(ashbaugh_Pairs)[0])
        
        ashbaugh_Indices_One = np.asarray([AA]*len(ashbaugh_Pairs))
        ashbaugh_Indices_Two = ashbaugh_Pairs[:,1]
        labels_One = trajectory_Particle_Labels[0,ashbaugh_Indices_One]
        labels_Two = trajectory_Particle_Labels[0,ashbaugh_Indices_Two]
        sigmas_One = sigmas[labels_One]
        sigmas_Two = sigmas[labels_Two]
        sigma_Averages = (sigmas_One + sigmas_Two)/2
        cutoffs = 2**(1/6)*sigma_Averages
        lambdas_One = lambdas[labels_One]
        lambdas_Two = lambdas[labels_Two]
        lambda_Averages = (lambdas_One + lambdas_Two)/2
        
        current_Distances = distances_Array[ashbaugh_Pairs[:,0],ashbaugh_Indices_Two]
        greater_Than_Threshold = np.where(current_Distances > cutoffs)
        lesser_Than_Threshold = np.where(current_Distances < cutoffs)
        hydrophobic_Forces[greater_Than_Threshold] = (48*LJ_Potential_Epsilon_Converted/current_Distances[greater_Than_Threshold])*((sigma_Averages[greater_Than_Threshold]/current_Distances[greater_Than_Threshold])**12 - 0.5*(sigma_Averages[greater_Than_Threshold]/current_Distances[greater_Than_Threshold])**6)
        hydrophobic_Forces[lesser_Than_Threshold] = lambda_Averages[lesser_Than_Threshold]*(48*LJ_Potential_Epsilon_Converted/current_Distances[lesser_Than_Threshold])*((sigma_Averages[lesser_Than_Threshold]/current_Distances[lesser_Than_Threshold])**12 - 0.5*(sigma_Averages[lesser_Than_Threshold]/current_Distances[lesser_Than_Threshold])**6)
        current_Hydrophobic_Forces_Array[0,ashbaugh_Indices_Two,ashbaugh_Pairs[:,0]] = hydrophobic_Forces  
            
        for column in range(AAs):
            average_Hydrophobic_Forces_Array[AA,column] = np.mean(current_Hydrophobic_Forces_Array[0,column,np.where(current_Hydrophobic_Forces_Array[0,column,:] <= 0)])
        average_Electrostatic_Forces_Array[AA,:] = np.mean(current_Electrostatic_Forces_Array, axis = 2)[0]
    
    np.save(output_Working_Directory_One + file.replace("Positions.npy","Electrostatic_Forces.npy"), average_Electrostatic_Forces_Array)
    np.save(output_Working_Directory_Two + file.replace("Positions.npy","Hydrophobic_Forces.npy"), average_Hydrophobic_Forces_Array)